In [1]:
import numpy as np
import numba as nb
from numba.typed import List
import pandas as pd
import itertools
from operator import itemgetter
import gc

from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof

In [2]:
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2

In [5]:
par_rock_volumes = [1e9, 1e8, 1e7, 1e6, 1e5, 1e4, 1e3, 1e2, 1e1]
minerals = ["Q", "P", "K", "B", "O", "A"]
par_rock_volume = 1e9
print(par_rock_volume)


1000000000.0


In [6]:
modal_mineralogy = np.array([0.30591989, 0.38159713, 0.26209888, 0.0188256 , 0.00799247, 0.02356603])

In [7]:
csds_CA_NS_means = np.array([0.309, 0.330, 0.244, 0.223, 0.120, 0.122])
csds_CA_NS_stds = np.array([0.823, 0.683, 0.817, 0.819, 0.554, 0.782])

In [8]:
sedgentest = SedGen(minerals, par_rock_volume, modal_mineralogy, csds_CA_NS_means, csds_CA_NS_stds, learning_rate = 10000000, timed = True, discretization_init = False)

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 177.1846 seconds
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 3 0 ... 0 0 0]
 Done in 4.0076 seconds
Counting interfaces...  Done in 0.0407 seconds
Correcting interface arrays for consistency...
too much Q 1
too few P -1
too few K -1
too much B 1
all good O 0
all good A 0
Initializing crystal size array... |Q|P|K|B|O|A| Done in 0.2551 seconds
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...

---SedGen model initialization finished succesfully---


In [9]:
sedgentest.simulated_volume
np.sum(sedgentest.simulated_volume)

array([3.05926919e+08, 3.81599202e+08, 2.62099208e+08, 1.88264299e+07,
       7.99550837e+06, 2.35678778e+07])

1000015144.2818536

In [14]:
sedgen_CA_NS_list = []
for i in range (len(par_rock_volumes)):
    print(par_rock_volumes[i])
    sedgen_CA_NS = SedGen (minerals, par_rock_volumes[i], modal_mineralogy, csds_CA_NS_means, csds_CA_NS_stds, learning_rate=10000, timed = False)
    sedgen_CA_NS_list.append(sedgen_CA_NS)
    

#now the model should have ran 9 times for different parent rock volumes, and the variable 'sedgen_CA_NS_list' should contain the output
#

1000000000.0
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|
Counting interfaces...
Correcting interface arrays for consistency...
too much Q 4
too few P -2
too few K -1
all good B 0
all good O 0
too few A -1
Initializing crystal size array... |Q|P|K|B|O|A|
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...
Initializing discretization for model's weathering...

---SedGen model initialization finished succesfully---
100000000.0
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|
Counting interfaces...
Correcting interface arrays for consistency...
all good Q 0
all good P 0
all good K 0
too few B -1
all good O 0
too much A 1
Initializin

IndexError: tuple index out of range

In [15]:
par_rock_volumes[0] * modal_mineralogy

array([3.0591989e+08, 3.8159713e+08, 2.6209888e+08, 1.8825600e+07,
       7.9924700e+06, 2.3566030e+07])

In [16]:
np.sum(sedgen_CA_NS_list[0].mcg)
sedgen_CA_NS_list[2].simulated_volume/1e7
np.sum(sedgen_CA_NS_list[2].simulated_volume)

0

array([0.30622172, 0.38260731, 0.2623736 , 0.01920425, 0.00884281,
       0.02386898])

10031186.72517989

In [28]:
for i in range (9):
    if i == 0:
        deler = 1e9
    elif i == 1:
        deler = 1e8
    elif i == 2:
        deler = 1e7
    elif i == 3:
        deler = 1e6
    elif i == 4:
        deler = 1e5
    elif i == 5:
        deler = 1e4
    elif i == 6:
        deler = 1e3
    elif i == 7:
        deler = 1e2
    elif i == 8:
        deler = 1e1
    ondergrens =  modal_mineralogy-(0.05*modal_mineralogy)
    bovengrens =  modal_mineralogy+(0.05*modal_mineralogy)
    voorwaarde1 = sedgen_CA_NS_list[i].simulated_volume/deler >= ondergrens
    voorwaarde2 = sedgen_CA_NS_list[i].simulated_volume/deler <= bovengrens
    if np.all(voorwaarde1 == True) and np.all(voorwaarde2 == True):
        print("De modale mineralogy komt overeen voor een parent rock volume van: ",deler, "vierkante mm.")
    else: 
        print("De modale mineralogy komt niet overeen voor een parent rock volume van:",deler, "vierkante mm. Het kritieke volume is dus:", deler * 10," vierkante mm.")
        break

De modale mineralogy komt overeen voor een parent rock volume van:  1000000000.0 vierkante mm.
De modale mineralogy komt overeen voor een parent rock volume van:  100000000.0 vierkante mm.
De modale mineralogy komt niet overeen voor een parent rock volume van: 10000000.0 vierkante mm. Het kritieke volume is dus: 100000000.0  vierkante mm.
